# Classificador de notícies

En aquesta pràctica, crearem un classificador de notícies utilitzant les tècniques de processament de llenguatge natural que hem vist a classe, centrades en la representació del text.

Utilitzarem el `dataset` [AG News](https://www.kaggle.com/amananandrai/ag-news-classification-dataset) que conté 1.000.000 de notícies de 4 categories diferents.

## Dataset

Per carregar el dataset, utilitzarem la llibreria `tensorflow_datasets` que ens permetrà carregar-lo de forma senzilla. Aquesta llibreria també ens permetrà fer la separació del dataset en conjunt d'entrenament i de test.

Per carregar el dataset, utilitzarem la funció `load('ag_news_subset')` de la llibreria `tensorflow_datasets`. Aquesta funció retorna un `tf.data.Dataset` que conté els exemples del dataset. Aquest objecte és similar a un `numpy.array` però conté més informació sobre el dataset. En concret, conté els exemples del dataset i la seva etiqueta. Per accedir als exemples i les etiquetes, utilitzarem els atributs `data` i `label` de l'objecte `tf.data.Dataset`.

## Preparació del dataset

Per instal·lar la llibreria `tensorflow_datasets`, executarem la següent cel·la. Aquesta llibreria conté molts datasets que poden ser útils per a la pràctica de l'assignatura.

In [1]:
# Instalem les llibreries necessàries

!pip install keras
!pip install tensorflow
!pip install tensorflow_datasets


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Using cached tensorflow_datasets-4.9.4-py3-none-any.whl (5.1 MB)
  Using cached dm_tree-0.1.8-cp311-cp311-macosx_10_9_x86_64.whl (115 kB)
  Using cached etils-1.6.0-py3-none-any.whl (144 kB)
  Using cached promise-2.3-py3-none-any.whl
  Using cached tensorflow_metadata-1.14.0-py3-none-any.whl (28 kB)
  Using cached fsspec-2023.12.2-py3-none-any.whl (168 kB)
  Using cached importlib_resources-6.1.1-py3-none-any.whl (33 kB)
  Using cached googleapis_common_protos-1.62.0-py2.py3-none-any.whl (228 kB)
  Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.24.1
    Uninstalling protobuf-4.24.1:
      Successfully uninstalled protobuf-4.24.1
ERROR: pip's dependency resolver does not currently

In [2]:
import keras
import tensorflow as tf
import tensorflow_datasets as tfds

# En aquest tutorial, entrenarem molts models. Per utilitzar la memòria de la GPU de forma eficient,
# configurarem tensorflow perquè creixi la memòria de la GPU quan sigui necessari.
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Carreguem el dataset. Es descarregarà automàticament i es guardarà a la carpeta
# /home/USUARI/tensorflow_datasets/ag_news_subset/1.0.0
dataset = tfds.load('ag_news_subset')

Automáticament, la funció `load` ha dividit el dataset en dos conjunts: un de train i un de test. Per accedir a aquests conjunts, utilitzarem els atributs `train` i `test` de l'objecte `dataset`. Aquests atributs són objectes `tf.data.Dataset` que contenen els exemples i les etiquetes del conjunt d'entrenament i de test. Per accedir als exemples i les etiquetes, utilitzarem els atributs `data` i `label` de l'objecte `tf.data.Dataset`.

In [3]:
# Separem el dataset en conjunt d'entrenament i de test
ds_train = dataset['train']
ds_test = dataset['test']
# Vejam quants exemples hi ha en cada conjunt
print('Nombre d\'exemples de train:', len(ds_train))
print('Nombre d\'exemples de test:', len(ds_test))

Nombre d'exemples de train: 120000
Nombre d'exemples de test: 7600


Imprimim els primers 5 exemples del conjunt d'entrenament. Com podem veure, cada exemple és una notícia i la seva etiqueta.

In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

# Imprimim els primers 5 exemples del conjunt d'entrenament
for w in ds_train.take(5):
    print(f"{w['label']} ({classes[w['label']]}) -> {w['title']} {w['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

2024-01-24 12:57:04.465042: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Representació del text

Per poder entrenar un model de xarxes neuronals, necessitem representar el text com a vectors de nombres. En aquesta pràctica, utilitzarem la representació Bag-of-Words (BoW) que consisteix en representar cada paraula com un nombre. Aquesta representació és molt senzilla i no té en compte l'ordre de les paraules ni la seva semàntica. Però és una representació que funciona prou bé en molts casos.

### Limitem el vocabulari

En el dataset, hi ha moltes paraules que apareixen poques vegades. Aquestes paraules no són útils per entrenar el model i només augmenten la dimensionalitat de la representació. Per això, limitarem el vocabulari a les 50.000 paraules més freqüents. 

Per fer aquest procés utilitzarem la capa `TextVectorization` de `keras`. Aquesta capa ens permetrà convertir el text en seqüències de nombres. A més, ens permetrà limitar el vocabulari i convertir les paraules a minúscules. Per fer-ho, utilitzarem els paràmetres `max_tokens` i `standardize` de la capa `TextVectorization`.

> Per construïr el vocabulari, utilitzarem solament 500 notícies del conjunt d'entrenament. Això és perquè el procés de construcció del vocabulari és molt lent i volem reduïr el temps d'execució del tutorial. En un cas real, utilitzaríem totes les notícies del conjunt d'entrenament. Asumim el risc de que alguna paraula del conjunt de test no estigui en el vocabulari i baixe l'accuracy del model; no deuria ser un problema greu de tota manera.


In [5]:
# Limitem el vocabulari a les 50.000 paraules més freqüents
tamany_vocabulari = 50000

# Creem la capa TextVectorization
# max_tokens: nombre màxim de paraules que tindrà el vocabulari
# standardize: funció que s'aplicarà a cada paraula per convertir-la a minúscules
# output_mode: 'int' perquè la capa retorni seqüències de nombres (per defecte)
vectorize_layer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=tamany_vocabulari,
    standardize='lower_and_strip_punctuation',
    output_mode='int'
)

# Actualitzem el vocabulari de la capa TextVectorization amb les paraules del conjunt d'entrenament
# Per fer-ho, primer creem un dataset amb les notícies del conjunt d'entrenament. Per reduïr el temps d'execució solament treballarem en 500 notícies
ds_text = ds_train.take(500).map(lambda article: article['title'] + ' ' + article['description'])

# Després, apliquem la capa TextVectorization a aquest dataset per construir el vocabulari
vectorize_layer.adapt(ds_text)

2024-01-24 12:57:05.568339: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


Ara ja podem accedir al vocabulari de la capa `TextVectorization` utilitzant l'atribut `get_vocabulary`. Aquest atribut retorna una llista amb les paraules del vocabulari. Com podem veure, la capa `TextVectorization` ha convertit les paraules a minúscules i ha eliminat els caràcters de puntuació.

In [6]:
# Imprimim les 10 primeres paraules del vocabulari
print(vectorize_layer.get_vocabulary()[:10])

# Imprimim el tamany del vocabulari
tamany_vocabulari = len(vectorize_layer.get_vocabulary())
print('Tamany del vocabulari:', tamany_vocabulari)

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Tamany del vocabulari: 5335


Utilitzant el vectoritzador, podem convertir un text en una seqüència de nombres. Vejam un exemple.

In [7]:
vectorize_layer('this is a test sentence')

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 35,  17,   4, 675,   1])>

## Representació Bag-of-Words

Encara que el significat de les paraules no és fàcil de deduir sense poder accedir al context, en alguns casos, la representació Bag-of-Words pot ser útil. Per exemple, en el text d'una notícia, la paraula `covid` pot ser un bon indicador que la notícia parla sobre la pandèmia de la COVID-19 i la paraula `snow` pot ser un bon indicador que la notícia parla sobre el temps atmosfèric.

De les tècniques clàssiques de vectorització de text, la més senzilla és la representació Bag-of-Words (BoW). En aquesta representació, cada paraula es representa com un nombre. Per convertir un text en una representació BoW, primer creem un vector amb tants zeros com paraules hi ha en el vocabulari. Després, per cada paraula del text, incrementem en 1 el valor de la posició corresponent al vector. Per exemple, si el text és `this sentence is a test sentence`, el vector resultant seria `[1, 2, 1, 1, 0, 0, 0, 0, 0, 0, ...]`.

Si recordem la representació one-hot, veurem que la representació BoW és molt similar. La diferència és que la representació one-hot serà una sèrie de vectors amb un sol 1 i la resta de valors a 0. En canvi, la representació BoW serà un vector amb tants 1 com vegades apareixi cada paraula. Podem considerar que la representació BoW seria la suma de vectors one-hot.

Per exemple, si el text és `this sentence is a test sentence`, el vector one-hot de la primera paraula seria `[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]` i el vector one-hot de la segona paraula seria `[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]`. La representació BoW seria la suma d'aquests dos vectors: `[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...]`.

Per generar una representació BoW, utilitzarem aquesta tècnica per convertir cada paraula en un vector one-hot i després sumarem tots els vectors. Per fer-ho, utilitzarem la funció `to_bow` que crearem a continuació. Aquesta funció rep un text i retorna un vector amb la representació BoW del text.

In [8]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorize_layer(text), tamany_vocabulari), axis=0)


to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

### Entrenament del model de classificació BoW

Per entrenar el model de classificació, utilitzarem la capa `TextVectorization` per convertir el text en una representació BoW. Després, utilitzarem una capa `Dense` per classificar el text. Aquesta capa és una xarxa neuronal totalment connectada amb una capa d'entrada i una capa de sortida. La capa d'entrada tindrà tants neurones com paraules hi hagi en el vocabulari i la capa de sortida tindrà tantes neurones com classes hi hagi en el problema. En aquest cas, la capa de sortida tindrà 4 neurones perquè el problema té 4 classes.

### Representació BoW

Convertim el text en la representació BoW utilitzant la funció `to_bow` que hem creat abans. Aquesta funció rep un text i retorna un vector amb la representació BoW del text.

Utilitzarem la funció `map` de `tf.data.Dataset` per aplicar la funció `to_bow` a cada exemple del dataset. Aquesta funció retorna un nou dataset amb els exemples transformats. Després, utilitzarem la funció `batch` per agrupar els exemples en lots de 128 exemples i optimitzar el procés d'entrenament.

In [9]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title'] + ' ' + x['description']), x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title'] + ' ' + x['description']), x['label'])).batch(batch_size)

### Model de classificació

Per crear el model de classificació, utilitzarem la classe `Sequential` de `keras`. Aquesta classe ens permet crear models seqüencials de forma senzilla. Per crear el model, li passarem una llista amb les capes que volem afegir al model. 

En aquest cas, el model tindrà una capa `Dense` amb 4 neurones perquè el problema té 4 classes i tantes neurones d'entrada com paraules hi hagi en el vocabulari (`input_shape=(tamany_vocabulari,)`). Aquesta capa tindrà una funció d'activació `softmax` perquè volem que la sortida del model sigui una distribució de probabilitats sobre les classes. 

Per entrenar el model, utilitzarem l'optimitzador `Adam` (ja que és un dels optimitzadors més utilitzats) i la funció de cost `SparseCategoricalCrossentropy` (ja que tenim un problema de classificació amb més de dues classes). Per compilar el model, utilitzarem la funció `compile` i li passarem l'optimitzador i la funció de cost. A més, li passarem la llista de mètriques que volem calcular durant l'entrenament. En aquest cas, volem calcular l'accuracy.

Per últim, entrenarem el model utilitzant la funció `fit` i li passarem el dataset d'entrenament i el dataset de validació. Aquesta funció entrenarà el model i calcularà l'accuracy del model en el dataset de validació a cada època. Així, podrem veure com evoluciona l'accuracy del model durant l'entrenament.

In [10]:
model = keras.Sequential([
    keras.layers.Dense(4, input_shape=(tamany_vocabulari,), activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(ds_train_bow, validation_data=ds_test_bow)

938/938 [==============================] - 101s 107ms/step - loss: 0.5926 - accuracy: 0.8492 - val_loss: 0.4160 - val_accuracy: 0.8784


El model ha aconseguit una accuracy de més de 0.86 en el conjunt d'entrenament; un nombre prou acceptable tenint en compte que hem simplificat el problema per reduïr el temps d'execució del tutorial. En un cas real, utilitzaríem totes les notícies del conjunt d'entrenament i el model seria més precís.

### Clasificador com a xarxa neuronal

Podem aprofitar que hem definit el vectoritzador com una capa de `keras` per crear un model de classificació com una xarxa neuronal completa. Aixó ens premetrà entrenar el model de classificació i el vectoritzador alhora; així com guardar i carregar el model de classificació i el vectoritzador en un sol fitxer.

Crearem un model amb les següents capes:

* La capa d'entrada agafrà un string amb el text de la notícia
* La capa `TextVectorization` convertirà el text en array d'enters
* La capa `one_hot` convertirà l'array d'enters en un array de vectors `one-hot`
* La capa `reduce_sum` sumarà els vectors one-hot per obtenir la representació BoW
* La capa `Dense` classificarà el text en una de les 4 categories

In [11]:
def extrau_text(x):
    # Concatenem el títol i la descripció de la notícia
    return x['title'] + x['description']


def dict_a_tupla(x):
    # Convertim el diccionari en una tupla amb el text i l'etiqueta
    return (extrau_text(x), x['label'])


# Creem el model
# La capa d'entrada serà un string amb el text de la notícia
inp = keras.Input(shape=(1,), dtype=tf.string)

# Vectorize_layer retorna un tensor amb un enter per cada paraula del text
enters = vectorize_layer(inp)

# Convertim el tensor d'enters en un tensor d'arrays one-hot
one_hots = tf.one_hot(enters, tamany_vocabulari)

# Sumem els vectors one-hot per obtenir la representació BoW
bow = tf.reduce_sum(one_hots, axis=1)

# Creem la capa de sortida amb 4 neurones i funció d'activació softmax
out = keras.layers.Dense(4, activation='softmax')(bow)

# Creem el model
model = keras.models.Model(inp, out)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, None)              0         
 ctorization)                                                    
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLa  (None, 5335)              0         
 mbda)                                                           
                                                                 
 dense_1 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21344 (83.38 KB)
Trainable params: 21344 (83.38

### Entrenament del model

Per entrenar el model, utilitzarem la funció `map` de `tf.data.Dataset` per aplicar la funció `dict_a_tupla` a cada exemple del dataset. Aquesta funció retorna un nou dataset amb els exemples transformats. Després, utilitzarem la funció `batch` per agrupar els exemples en lots de 128 exemples i optimitzar el procés d'entrenament.

In [12]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train.map(dict_a_tupla).batch(batch_size), validation_data=ds_test.map(dict_a_tupla).batch(batch_size))

938/938 [==============================] - 154s 164ms/step - loss: 0.6205 - acc: 0.8381 - val_loss: 0.4456 - val_acc: 0.8658


### Generar els Vectors BoW automàticament

En el model anterior, hem creat manualment la representació BoW. Això ens ha permès entendre com funciona la representació BoW i com es pot implementar en `keras`. Però, en la pràctica, no cal que creem manualment la representació BoW perquè `keras` ja té una opcio per fer-ho: el paràmetre `output_mode="count"` de la capa `TextVectorization`.

In [13]:
# Creem la xarxa amb la capa TextVectorization
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=tamany_vocabulari, output_mode='count'),
    keras.layers.Dense(4, input_shape=(tamany_vocabulari,), activation='softmax')
])

# Actualitzem el vocabulari de la capa TextVectorization amb les paraules del conjunt d'entrenament
model.layers[0].adapt(ds_train.take(500).map(extrau_text))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train.map(dict_a_tupla).batch(batch_size), validation_data=ds_test.map(dict_a_tupla).batch(batch_size))

model.summary()

938/938 [==============================] - 12s 12ms/step - loss: 0.6142 - acc: 0.8421 - val_loss: 0.4369 - val_acc: 0.8700
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (Text  (None, 5335)              0         
 Vectorization)                                                  
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21344 (83.38 KB)
Trainable params: 21344 (83.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### N-grames

La representació BoW és molt senzilla i no té en compte l'ordre de les paraules ni la seva semàntica. Per això, en alguns casos, la representació BoW pot ser millorada utilitzant n-grames. Un n-grames és una seqüència de n paraules consecutives. Per exemple, en el text `this sentence is a test sentence`, els 2-grames serien `this sentence`, `sentence is`, `is a`, `a test` i `test sentence`.

Per generar una representació BoW utilitzant n-grames, veurem una altra forma de generar la representació BoW: `CountVectorizer`. Aquesta funció rep una llista de textos i retorna una matriu on cada fila és un text i cada columna és una paraula del vocabulari. El valor de cada cèl·lula és el nombre de vegades que apareix la paraula en el text. Per exemple, si el text és `this sentence is a test sentence`, el vector BoW seria `[1, 2, 1, 1, 0, 0, 0, 0, 0, 0, ...]` i la matriu BoW seria `[[1, 2, 1, 1, 0, 0, 0, 0, 0, 0, ...]]`.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
    'I like hot dogs.',
    'The dog ran fast.',
    'Its hot outside.',
]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n", bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

El major problema dels `n-grames` és que el nombre de n-grames creix molt ràpidament amb n. Per exemple, si tenim un vocabulari de 50.000 paraules, el nombre de 2-grames serà de 2.500.000.000. Per això, en la pràctica, s'han de combinar o substituir per altres tècniques de reducció de dimensionalitat com ara `Word2Vec` o `GloVe`.

## Representació TF-IDF

Una altra representació molt utilitzada en el processament de llenguatge natural és la representació TF-IDF. Aquesta representació té en compte la freqüència de les paraules en el text i en el conjunt de documents. Això permet donar més importància a les paraules que defineixen un text.

Dit d'una altra manera, si una paraula apareix moltes vegades en un text però també apareix en altres textos, aquesta paraula no aporta molta informació sobre el text (`and`). En canvi, si una paraula apareix moltes vegades en un text i no apareix tant en altres textos (`covid`), aquesta paraula aporta molta informació sobre el text.

Per utilitzar la representació TF-IDF, utilitzarem el paràmetre `output_mode="tf-idf"` de la capa `TextVectorization`. Fer la prova per veure sl'accuracy del model millora un poc.

In [15]:
# Creem la xarxa amb la capa TextVectorization
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=tamany_vocabulari, output_mode='tf-idf'),
    keras.layers.Dense(4, input_shape=(tamany_vocabulari,), activation='softmax')
])

# Actualitzem el vocabulari de la capa TextVectorization amb les paraules del conjunt d'entrenament
model.layers[0].adapt(ds_train.take(500).map(extrau_text))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(ds_train.map(dict_a_tupla).batch(batch_size), validation_data=ds_test.map(dict_a_tupla).batch(batch_size))

model.summary()

938/938 [==============================] - 12s 11ms/step - loss: 0.4270 - acc: 0.8660 - val_loss: 0.3544 - val_acc: 0.8849
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (Text  (None, 5335)              1         
 Vectorization)                                                  
                                                                 
 dense_3 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21345 (83.38 KB)
Trainable params: 21344 (83.38 KB)
Non-trainable params: 1 (8.00 Byte)
_________________________________________________________________


## Representació Word2Vec

La representació Word2Vec és una representació molt utilitzada en el processament de llenguatge natural. Aquesta representació té en compte el context de les paraules i permet fer operacions amb les paraules. Per exemple, si restem el vector de la paraula `king` i sumem el vector de la paraula `woman`, obtindrem un vector que serà molt similar al vector de la paraula `queen`.

Per generar la representació Word2Vec, utilitzarem la llibreria `gensim`. Aquesta llibreria conté molts models de representació de paraules. En aquest cas, utilitzarem el model `word2vec-google-news-300` que conté la representació Word2Vec de 3 milions de paraules i frases. 

> La primera vegada que s'executi aquesta cel·la, la funció `load` descarregarà el model d'1.5GB. Això pot trigar uns minuts. Un cop descarregat, el model es guardarà a la carpeta `/home/USUARI/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz` i no caldrà descarregar-lo de nou.
> Aquesta funció retorna un objecte `KeyedVectors` que conté la representació Word2Vec.

In [ ]:
import gensim.downloader as api

w2v = api.load('word2vec-google-news-300')

[=-------------------------------------------------] 2.9% 47.5/1662.8MB downloaded

Ara ja podem accedir a la representació Word2Vec de cada paraula. Per exemple, per accedir a la representació de la paraula `king`, utilitzarem la funció `get_vector` de l'objecte `KeyedVectors`.

In [ ]:
w2v.get_vector('king')

També podem accedir a les paraules més similars a una paraula. Per exemple, per accedir a les paraules més similars a la paraula `king`, utilitzarem la funció `most_similar` de l'objecte `KeyedVectors`.

In [ ]:
for w, p in w2v.most_similar('king'):
    print(f"{w} -> {p}")

El més interessant de la representació Word2Vec és que els vectors tenen una estructura matemàtica que permet fer operacions amb les paraules. Per exemple, si restem el vector de la paraula `king` i sumem el vector de la paraula `woman`, obtindrem un vector que serà molt similar al vector de la paraula `queen`.

$$ KING - MAN + WOMAN = QUEEN $$

Per fer aquesta operació, utilitzarem la funció `most_similar` de l'objecte `KeyedVectors` i li passarem els vectors de les paraules `king`, `woman` i `man`. Aquesta funció retornarà una llista amb les paraules més similars al vector resultant. Com podem veure, la paraula més similar és `queen`.

In [ ]:
w2v.most_similar(positive=['king', 'woman'], negative=['man'])[0]

In [ ]:
from keras.src.layers import Embedding


def gensim_to_keras_embedding(model, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [ ]:
model = keras.models.Sequential([
    vectorize_layer,
    gensim_to_keras_embedding(w2v, train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(dict_a_tupla).batch(128),validation_data=ds_test.map(dict_a_tupla).batch(128),epochs=5)

El resultat no es molt bo. Això és perquè el model Word2Vec que hem utilitzat no té les paraules que apareixen en el dataset. Per exemple, si busquem la paraula `covid`, veurem que no apareix en el model.

Per solucionar aquest problema hauriem d'utilitzar un model Word2Vec entrenat amb les paraules del dataset. Però això és molt lent i no ho farem en aquest tutorial.